In [ ]:
!pip uninstall -y cupy cupy-cuda11x cupy-cuda112 cupy-cuda113 cupy-cuda115

In [ ]:
!pip install cupy-cuda11x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 MB 6.8 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!nvidia-smi

Mon Jun 16 16:51:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip uninstall -y cupy cupy-cuda11x cupy-cuda12x cupy-cuda120 cupy-cuda121 cupy-cuda122 cupy-cuda124

Found existing installation: cupy-cuda12x 13.3.0
Uninstalling cupy-cuda12x-13.3.0:
  Successfully uninstalled cupy-cuda12x-13.3.0


In [ ]:
!pip install cupy-cuda12x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 MB 8.8 MB/s eta 0:00:00


In [ ]:
import cupy as cp
print("GPU count:", cp.cuda.runtime.getDeviceCount())

GPU count: 1


In [ ]:
import cupy as cp
import time

def initialize_road(L, density, vmax):
    return cp.where(
        cp.random.random(L) < density,
        cp.random.randint(0, vmax+1, size=L, dtype=cp.int32),
        cp.int32(-1)
    )

def simulate_nasch_gpu(L, density, vmax, p_slow, steps):
    road = initialize_road(L, density, vmax)
    for _ in range(steps):
        veh_mask = road >= 0
        road[(veh_mask)&(road<vmax)] += 1

        empty = road == -1
        gap   = cp.zeros_like(road)
        for shift in [1,2,4,8,16,32,64,128,256,512,1024,2048]:
            e   = cp.roll(empty, -shift)
            gap = cp.where(e, gap+shift, gap)

        idxs = cp.where(veh_mask)[0]
        vs   = road[idxs]
        gs   = gap[idxs]
        road[idxs] = cp.minimum(vs, gs)

        rand = cp.random.random(road.size)
        slow = (road>0)&(rand<p_slow)
        road[slow] -= 1

        new_road = cp.full_like(road, -1)
        idxs     = cp.where(road>=0)[0]
        dst      = (idxs + road[idxs]) % L
        new_road[dst] = road[idxs]
        road = new_road

    return road

# Parámetros
L       = 500_000   # longitud de la carretera
density = 0.5       # densidad de vehículos
vmax    = 10        # velocidad máxima
p_slow  = 0.3       # probabilidad de frenado aleatorio
steps   = 500       # pasos de simulación

# Ejecución y medida de tiempo
start = time.time()
road_final = simulate_nasch_gpu(L, density, vmax, p_slow, steps)
cp.cuda.Stream.null.synchronize()
print("Tiempo GPU:", time.time() - start)

Tiempo GPU: 5.56258225440979


In [ ]:
# === Celda 1: Definición de funciones y simulación GPU ===

import cupy as cp
import time

def initialize_road(L, density, vmax):
    """
    Inicializa la carretera en GPU:
      -1 = celda vacía
      0…vmax = velocidad de un vehículo
    """
    return cp.where(
        cp.random.random(L) < density,
        cp.random.randint(0, vmax + 1, size=L, dtype=cp.int32),
        cp.int32(-1)
    )

def simulate_nasch_gpu(L, density, vmax, p_slow, steps):
    """
    Simula el modelo Nagel–Schreckenberg en GPU usando CuPy.
    """
    road = initialize_road(L, density, vmax)
    for _ in range(steps):
        # 1) Aceleración
        veh_mask = road >= 0
        road[veh_mask & (road < vmax)] += 1

        # 2) Cálculo de huecos (gap)
        empty = road == -1
        gap = cp.zeros_like(road)
        for shift in [1,2,4,8,16,32,64,128,256,512,1024,2048]:
            e   = cp.roll(empty, -shift)
            gap = cp.where(e, gap + shift, gap)

        # 3) Frenado según gap
        idxs = cp.where(veh_mask)[0]
        vels = road[idxs]
        gaps = gap[idxs]
        road[idxs] = cp.minimum(vels, gaps)

        # 4) Desaceleración aleatoria
        rand      = cp.random.random(road.size)
        slow_mask = (road > 0) & (rand < p_slow)
        road[slow_mask] -= 1

        # 5) Movimiento
        new_road = cp.full_like(road, -1)
        idxs     = cp.where(road >= 0)[0]
        vels     = road[idxs]
        dests    = (idxs + vels) % L
        new_road[dests] = vels
        road = new_road

    return road

def run_experiment(params):
    """
    Ejecuta un experimento (L, steps) y mide su tiempo en GPU.
    """
    L, steps = params
    density, vmax, p_slow = 0.5, 10, 0.3
    start = time.time()
    simulate_nasch_gpu(L, density, vmax, p_slow, steps)
    cp.cuda.Stream.null.synchronize()
    elapsed = time.time() - start
    return {"L": L, "steps": steps, "time_s": elapsed}


In [ ]:
# === Celda 2: Lanzamiento concurrente de experimentos via hilos ===

from concurrent.futures import ThreadPoolExecutor, as_completed

# Tu lista de experimentos
experiments = [
    (100_000, 500),
    (500_000, 500),
    (1_000_000, 500),
    (5_000_000, 500),
    (10_000_000, 500),
    (15_000_000, 500),
    (20_000_000, 500),
    (25_000_000, 500),
]

results = []
# Pon tantos hilos como quieras; con GPU no sirve de mucho poner muchos,
# pero 2–4 suele estar bien para solapar lanzamiento y fetch de resultados.
with ThreadPoolExecutor(max_workers=2) as executor:
    futures = { executor.submit(run_experiment, p): p for p in experiments }
    for fut in as_completed(futures):
        res = fut.result()
        print(f"L={res['L']:,}, steps={res['steps']} → {res['time_s']:.3f} s")
        results.append(res)

print("\nResumen ordenado:")
for r in sorted(results, key=lambda x:(x["L"], x["steps"])):
    print(f" • L={r['L']:,}, steps={r['steps']:<4} → {r['time_s']:.3f} s")


L=100,000, steps=500 → 6.200 s
L=500,000, steps=500 → 6.755 s
L=1,000,000, steps=500 → 7.173 s
L=5,000,000, steps=500 → 10.516 s
L=10,000,000, steps=500 → 24.827 s
L=15,000,000, steps=500 → 31.794 s
L=20,000,000, steps=500 → 47.520 s
L=25,000,000, steps=500 → 44.363 s

Resumen ordenado:
 • L=100,000, steps=500  → 6.200 s
 • L=500,000, steps=500  → 6.755 s
 • L=1,000,000, steps=500  → 7.173 s
 • L=5,000,000, steps=500  → 10.516 s
 • L=10,000,000, steps=500  → 24.827 s
 • L=15,000,000, steps=500  → 31.794 s
 • L=20,000,000, steps=500  → 47.520 s
 • L=25,000,000, steps=500  → 44.363 s
